## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  

In [1]:
# Testing the Lambda code
import json

url1 = 'https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg'
url2 = 'https://upload.wikimedia.org/wikipedia/commons/1/18/Vombatus_ursinus_-Maria_Island_National_Park.jpg'
event_dict = {
    "url": url1
}

event = json.dumps(event_dict)
print(event)

{"url": "https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg"}


In [2]:
import lambda_function
# Lambda takes care of converting the JSON to Python dict,
# so for local testing we send the dict instead
lambda_function.lambda_handler(event_dict, None)

[0.5814766883850098]

## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.10
COPY bees-wasps-v2.tflite .
```

And pushed it to [`agrigorev/zoomcamp-bees-wasps:v2`](https://hub.docker.com/r/agrigorev/zoomcamp-bees-wasps/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.
* The version of Python is 3.10, so you need to use the right wheel for 
  TF-Lite. For Tensorflow 2.14.0, it's https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.14.0-cp310-cp310-linux_x86_64.whl


## Question 5

Download the base image `agrigorev/zoomcamp-bees-wasps:v2`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 162 Mb
* 362 Mb
* 662 Mb
* 962 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.

In [14]:
!docker images agrigorev/zoomcamp-bees-wasps:v2

REPOSITORY                      TAG       IMAGE ID       CREATED      SIZE
agrigorev/zoomcamp-bees-wasps   v2        b9f6c13de368   4 days ago   662MB


Answer: `662 MB`

## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `bees-wasps-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference).

Now run the container locally.

Score this image: https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg

What's the output from the model?

* 0.2453
* 0.4453
* 0.6453
* 0.8453

> Build the Docker image with:

`docker build -f Dockerfile -t bees-wasps-lambda .`

> Run the image with the following:

`docker run -it --rm -p 8080:8080 bees-wasps-lambda:latest`

In [23]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations' #docker
data = {'url': 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'}


In [24]:
requests.post(url, json=data).json()

[0.4453350603580475]

Answer: `0.4453`

## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.

In [26]:
import requests

url='https://04eq8n9z4h.execute-api.us-east-1.amazonaws.com/test/predict' #api-gateway
data = {'url': 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'}

In [27]:
requests.post(url, json=data).json()

[0.4453350603580475]

## Publishing to Docker hub

This is just for reference, this is how we published our image to Docker hub:

```bash
docker build -f Dockerfile -t bees-wasps-lambda .
docker tag bees-wasps-lambda   mkahraman82/zoomcamp-bees-wasps:v2
docker push mkahraman82/zoomcamp-bees-wasps:v2
```
